# R23 - Molecular Dynamics - Dimensionality Reduction

## Introduction

Molecular Dynamics of large molecular systems, as solvated proteins, are usually challenging due to the huge number of degrees of freedom (generally, $3N -3$, where $N$ is the number of atoms). In extreme scenarios, we might work with millions of degrees of freedom. Therefore, how can we extract useful information from such a big amount of data?

Although solvated proteins and other molecular systems are very big and contain many atoms, the motions of these atoms are usually restrained due to different factors: molecular bonds, local stable conformations (e.g. secondary structure), global conformations (e.g. tertiary structure), etc. Therefore, it is possible that a space with lower dimensionality is still able to describe most of the movements that we observe — in other words, particles are correlated, so we don't need to know how each of them moves at all times.

The easiest path for trajectory dimensionality reduction is the linear path: using the principal component analysis on the 3D cartesian coordinates to find a set of orthogonal motions that represent how our particles move. If there are linear patterns in our trajectory, the PCA should be able to recover those as main components with large associated variances . If there are no clear patterns, then the PCA should provide a set of modes with very similar variances.

The PCA is computed from the diagonalization of the covariance matrix. If our trajectory over time generates a matrix A that is [t, 3*N], then we would compute the covariance of this matrix as:

$$ C = \frac{1}{3 * N}AA^{T} $$

The diagonalization of this matrix should provide a matrix P with all the modes, and a list of eigenvalues or variance-associated ratios that show how much does each of these modes represent the overall trajectory.

Luckily for us, we don't need to compute the PCA manually, we can just rely on the usage of either sklearn PCA implementation or in the usage of the essential dynamics module from MDAnalysis.



In [ ]:
md_reference_data = None
upstream = None
product = None

## Essential Dynamics - Performing dimensionality reduction through PCA

In [ ]:
import json
import prody as pdy
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
import MDAnalysis as mda
import seaborn as sns
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as sk_pca
import matplotlib.gridspec as gridspec
from MDAnalysis.analysis import pca as md_pca # We alias the funciton to avoid mistunderstandings
import MDAnalysis.transformations as trans
from MDAnalysis.analysis import align, rms, distances
from mdtools.mapping import map_alignment_to_structure, align_structure_sequences

from Bio import AlignIO

from Bio.SeqUtils import seq1
from Bio.SeqIO import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq

from tqdm import tqdm

In [ ]:
def trajectory_to_pc(trajectory, selection):
    ag = trajectory.select_atoms(selection)
    coordinates = np.stack([ag.positions.reshape(-1) for frame in trajectory.trajectory])
    coordinates = StandardScaler(with_std=False).fit_transform(coordinates)
    sk_pca_object = sk_pca(n_components=10)
    sk_pca_results = sk_pca_object.fit_transform(coordinates)
    return sk_pca_object.explained_variance_ratio_, sk_pca_results

In [ ]:
def pca_trajectory(pdb, dcd_list):
    trajectory = mda.Universe(
        pdb, 
        *dcd_list
    )
    ag = trajectory.select_atoms('name CA')
    coordinates = np.stack([ag.positions.reshape(-1) for frame in trajectory.trajectory])
    coordinates = StandardScaler(with_std=False).fit_transform(coordinates)
    sk_pca_object = sk_pca(n_components=10)
    sk_pca_results = sk_pca_object.fit_transform(coordinates)
    return sk_pca_results
    
def get_single_trajectory(trajectory, code, path, include_ssu=True):
    
    start_num_lsu = trajectory.select_atoms('name CA and segid A')[0].resid
    end_num_lsu = trajectory.select_atoms('name CA and segid A')[-1].resid
    if include_ssu:
        try:
            start_num_ssu = trajectory.select_atoms('name CA and segid B')[0].resid
            end_num_ssu = trajectory.select_atoms('name CA and segid B')[-1].resid
        except IndexError:
            include_ssu = False
    else:
        pass
    if include_ssu:
        ag = trajectory.select_atoms(
            'name CA and (segid A or segid E or segid I or segid M or segid C or segid G or segid K or segid O) and resid ' + str(start_num_lsu + 10) + \
            '-' + str(end_num_lsu - 10), 'name CA and (segid B or segid F or segid J or segid N or segid D or segid H or segid L or segid P) and resid ' + \
            str(start_num_ssu + 10) + '-' + str(end_num_ssu - 10))
    else:
        ag = trajectory.select_atoms('name CA and (segid A or segid E or segid I or segid M or segid C or segid G or segid K or segid O) and resid ' + \
            str(start_num_lsu + 10) + '-' + str(end_num_lsu - 10))
    coordinates = np.stack([ag.positions.reshape(-1) for frame in trajectory.trajectory])
    coordinates = StandardScaler(with_std=False).fit_transform(coordinates)
    sk_pca_object = sk_pca(n_components=10)
    sk_pca_results = sk_pca_object.fit_transform(coordinates)
    components = sk_pca_object.components_.copy()
    
    return sk_pca_results, components, sk_pca_object


def load_trajectory(code, data, path):
    
    """
    loads the following files
    - an MDAnalysis trajectory with the dried (water-removed) dcd
    - an MDAnalysis Universe with the starting PDB (jic)
    - an prody with the starting PDB (pretty useful when MDAnalysis fails)
    """
    return dict(
        code=code,
        trajectory_dry = mda.Universe(
            path + data['dry_pdb'], 
            path + data['dry_dcd'], 
            frames='all', in_memory=True
        ),
        reference_pdy=pdy.parsePDB(path + data['dry_pdb']),
        reference=mda.Universe(
            path + data['dry_pdb']
        ),
        time=data['time']
    )

def map_to_protein(code, components, ref):
    component_norm = np.linalg.norm(components.reshape([10, -1, 3]), axis=2)    # The [10, -1, 3] reshapes the components data into a 10 x X x 3 array, where -1 refers to the total number of residues/atoms that exist in the protein
    start_num_lsu = ref.select_atoms('name CA and segid A')[0].resid
    end_num_lsu = ref.select_atoms('name CA and segid A')[-1].resid
    ref.add_TopologyAttr('tempfactors')

    for i, res in enumerate(ref.select_atoms('protein and resid ' + str(start_num_lsu + 10) + '-' + str(end_num_lsu - 10)).residues):
        res.atoms.tempfactors = component_norm[0, i]/(component_norm[0, :].max())

    ref.select_atoms('protein').write(f'./output/{code}.pca-component0.pdb')


## Other Trajectories with single replicates
- 10 residue truncations are implemented at the N & C-terminal

In [ ]:
path = '../../../simulations/'
exp01_md = []
for key, items in md_reference_data.items():
    print(f"-- code {key}", end='')
    exp01_md.append(load_trajectory(
        code=key, data=items, path=path
    ))
    print(f" loaded")
exp01_md = pd.DataFrame.from_records(exp01_md).set_index('code')

In [ ]:
pca_out = []
for key, item in exp01_md.iterrows():

    lsussu_pca, lsussu_components, lsussu_pca_object = get_single_trajectory(item['trajectory_dry'], key, '../simulations/', include_ssu=True)
    lsu_pca, lsu_components, lsu_pca_object = get_single_trajectory(item['trajectory_dry'], key, '../simulations/', include_ssu=False)
    pca_out.append(dict(
        code=key, 
        lsussu_pca=lsussu_pca, lsussu_variance_ratio=lsussu_pca_object.explained_variance_ratio_,
        lsu_pca=lsu_pca, lsu_variance_ratio=lsu_pca_object.explained_variance_ratio_,
    ))
    # map_to_protein(key, lsussu_components, item['trajectory_dry'])

pca_out = pd.DataFrame.from_records(pca_out)

### PCA-plots for the motions of LSU-only

In [ ]:
for index, items in pca_out.iterrows():
    pca = np.array(items['lsu_pca'])
    g = sns.jointplot(x=pca[:, 0], y=pca[:, 1], kind='kde', joint_kws={"fill":True}, cmap='Spectral')
    g.plot_joint(sns.scatterplot, color="black", alpha=0.1)
    g.fig.suptitle(items['code'])

### Plotting the explained variance ratio with each PC

In [ ]:
for index, items in pca_out.iterrows():
    sns.lineplot(x=np.arange(len(items['lsu_variance_ratio'])), y=np.array(items['lsu_variance_ratio']), marker='o', label=items['code'])

### PCA-plots for the motions of both LSU and SSU

In [ ]:
for index, items in pca_out.iterrows():
    pca = np.array(items['lsussu_pca'])
    g = sns.jointplot(x=pca[:, 0], y=pca[:, 1], kind='kde', joint_kws={"fill":True}, cmap='Spectral')
    g.plot_joint(sns.scatterplot, color="black", alpha=0.1)
    g.fig.suptitle(items['code'])

In [ ]:
pca_out.to_json(product['singles_pca'], orient='records')

### 5 independent replicates of the PCA-plots

In [ ]:
# md05_ref_data = json.load(open('./md_simulations_early_branch_rep5.json', 'r'))

# path = '../../../simulations/'
# rep_5_md = []
# for key, items in md05_ref_data.items():
#     print(f"-- code {key}", end='')
#     rep_5_md.append(load_trajectory(
#         code=key, data=items, path=path
#     ))
#     print(f" loaded")
# rep_5_md = pd.DataFrame.from_records(rep_5_md).set_index('code')

In [ ]:
# rep5_pca_out = []
# for key, item in rep_5_md.iterrows():
#     lsu_pca, lsu_components, lsu_pca_object = get_single_trajectory(item['trajectory_dry'], key, '../simulations/', include_ssu=False)
#     rep5_pca_out.append(dict(
#         code=key, 
#         lsu_pca=lsu_pca, lsu_variance_ratio=lsu_pca_object.explained_variance_ratio_,
#     ))

# rep5_pca_out = pd.DataFrame.from_records(rep5_pca_out)

In [ ]:
# for index, items in rep5_pca_out.iterrows():
#     pca = np.array(items['lsu_pca'])
#     g = sns.jointplot(x=pca[:, 0], y=pca[:, 1], kind='kde', joint_kws={"fill":True}, cmap='Spectral')
#     g.plot_joint(sns.scatterplot, color="black", alpha=0.1)
#     g.fig.suptitle(items['code'])

## Essential Collective Dynamics

In [ ]:
def load_trajectory(code, data, path):
    return dict(
        code=code,
        trajectory_dry = mda.Universe(
            path + data['dry_pdb'], 
            path + data['dry_dcd'], 
            frames='all'
        ),
        reference_pdy=pdy.parsePDB(path + data['dry_pdb']),
        reference=mda.Universe(
            path + data['dry_pdb'], in_memory=True
        ),
        time=data['time']
    )

In [ ]:
def residues2sequence(residues):
    out = ""
    for res in residues:
        resname = res.resname.replace('KCX', 'LYS')

        out += seq1(resname)
    return out

def get_sequences(ref, chains):
    sequence_bag = []
    for chain in chains:
        sequence_bag.append(
            (  
                chain,
                residues2sequence(ref.select_atoms(f'protein and chainID {chain}').residues)
            )
            
        )
    return sequence_bag

In [ ]:
path = '../../../simulations/'
md_reference_data = json.load(open('./md_simulations.json', 'r'))
exp01_md = []
for key, items in md_reference_data.items():
    print(f"-- code {key}", end='')
    exp01_md.append(load_trajectory(
        code=key, data=items, path=path
    ))
    print(f" loaded")
exp01_md = pd.DataFrame.from_records(exp01_md).set_index('code')
a = align.AlignTraj(exp01_md.loc['ancip'].trajectory_dry, exp01_md.loc['ancip'].reference, in_memory=True, select='name CA')
a.run()

In [ ]:
exp01_md['rbcl_seq'] = exp01_md['reference'].apply(lambda x: get_sequences(x, chains='ACEGIKMO'))
exp01_md['rbcl_seq']

In [ ]:
seq_list = []
for key, item in exp01_md['rbcl_seq'].items():
    for chain, seq in item:
        seq_list.append(SeqRecord(seq=Seq(seq), id='{:s}_{:s}'.format(key, chain)))
SeqIO.write(seq_list, './output/sequence.fasta', 'fasta')

In [ ]:
!mafft --maxiterate 1000 --localpair './output/sequence.fasta' > './output/sequence.aligned.fasta'

In [ ]:
aligned_sequences = list(pdy.MSAFile(
    './output/sequence.aligned.fasta', format='fasta') # TODO Fix this
)

aligment_matrix = np.stack([item.getArray() for item in aligned_sequences])
conserved_positions = []
for i in range(aligment_matrix.shape[1]):
    if b'-' not in np.unique(aligment_matrix[:, i]).tolist():
        conserved_positions.append(i)

In [ ]:
def remove_gaps(sequence):
    for i, tokken in enumerate(sequence):
        if tokken != '-':
            yield i, tokken

def conserved2bfactor(sequence, conserved, chain, structure):
    # structure.add_TopologyAttr('tempfactors')
    residues = structure.select_atoms(f'protein and chainID {chain}').residues
    for (i, tokken), res in zip(remove_gaps(sequence), residues):
        
        if tokken != seq1(res.resname) and res.resname != "HIE":
            # print(res.resnum, tokken, seq1(res.resname))
            pass
        if i in conserved:
            try:
                print(res.resnum, tokken, seq1(res.resname))
                res.atoms.tempfactors = 1.0
            except AttributeError:
                print(f"{i} {tokken} {res.resname}")
        else:
            res.atoms.tempfactors = 0.0

In [ ]:
for seq in aligned_sequences:
    key = seq.getLabel()
    protein = key.split()[0].split('_')[0]
    chain = key.split()[0].split('_')[1]
    item = exp01_md.loc[protein]
    print(protein, chain)
    conserved2bfactor(
        str(seq), conserved_positions, chain, item.trajectory_dry
    )

In [ ]:
positions_buffer = []
label_buffer = []
frame_buffer = []
scaler = StandardScaler(with_std=False)

for key, item in exp01_md.iterrows():
    protein_positions_buffer = []
    for i, frame in enumerate(item.trajectory_dry.trajectory):
        tmp_positions = []
        for chain in 'ACEIGKMO':
            g = item.trajectory_dry.select_atoms(f'protein and prop tempfactor > 0.0 and name CA and chainID {chain}')
            positions = g.positions
            tmp_positions.append(positions)
            
        tmp_positions = np.stack(tmp_positions).reshape(-1)
        protein_positions_buffer.append(tmp_positions)
        label_buffer.append(key)
        frame_buffer.append(i)
    
    # protein_positions_buffer = scaler.fit_transform(np.stack(protein_positions_buffer))
    positions_buffer.append(np.stack(protein_positions_buffer))

positions_buffer = np.concatenate(positions_buffer)
# label_buffer = label_buffer
frame_buffer = np.array(frame_buffer)

In [ ]:
positions_buffer.shape

In [ ]:
plt.imshow(positions_buffer)

In [ ]:
pca = sk_pca(n_components=10, whiten=False)
scaler = StandardScaler(with_mean=True, with_std=False)

X = scaler.fit_transform(positions_buffer)
X = pca.fit_transform(X)

In [ ]:
fig, ax = plt.subplots(1)
fig.set_size_inches(3, 3)
ax.plot(np.arange(1, 11), pca.explained_variance_ratio_.cumsum(), marker='o', color='black')
ax.set_xlabel('Eigenvector index')
ax.set_ylabel('Variance explained (%)')
ax.set_ylim(None, 1.0)
fig.tight_layout()

In [ ]:
np.savetxt(
    product['joint_pca_variance'],
    pca.explained_variance_ratio_.cumsum()
)

In [ ]:
md_pca = pd.DataFrame.from_dict(dict(pca_x=X[:,0], pca_y=X[:, 1], pca_z=X[:, 2], time=frame_buffer, label=label_buffer))
md_pca.to_json(product['joint_pca'])

In [ ]:
g = sns.pairplot(md_pca[['pca_x', 'pca_y', 'pca_z', 'label']], hue='label', plot_kws={"alpha":0.5})

In [ ]:
g1 = sns.pairplot(md_pca[['pca_x', 'pca_y', 'pca_z', 'time']], hue='time', diag_kind=None)